# RNN (Recurrent Neural Network) 으로 텍스트 분류하기

In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from konlpy.tag import Okt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 데이터 미리보기 및 요약

In [27]:
df = pd.read_csv("다산콜재단.csv")

In [28]:
df.head()

,번호,분류,제목,내용,내용번호,문서
0,2645,복지,아빠 육아휴직 장려금,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...,23522464,아빠 육아휴직 장려금아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려...
1,2644,경제,[서울산업진흥원] 서울메이드란?,서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해...,23194045,[서울산업진흥원] 서울메이드란?서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하...
2,2642,복지,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",광진맘택시 운영임산부영아 양육가정 전용 택시 업무개요 교통약자인 임산부와 영아가정...,22904492,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)광진맘택시 운영임산부영아 양육가정 ..."
3,2641,복지,마포 뇌병변장애인 비전센터,마포 뇌병변장애인 비전센터 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학...,22477798,마포 뇌병변장애인 비전센터마포 뇌병변장애인 비전센터 마포뇌병변장애인 비전센터 운영 ...
4,2640,행정,2021년도 중1·고1 신입생 입학준비금 지원,년도 중고 신입생 입학준비금 지원 업무개요 서울시는 전국 최초로 년도부터 개 자...,22227896,2021년도 중1·고1 신입생 입학준비금 지원년도 중고 신입생 입학준비금 지원 업...


In [29]:
# value_counts()로 분류별 빈도수 확인
df["분류"].value_counts()

행정    1098
경제     823
복지     217
Name: 분류, dtype: int64

In [30]:
df.shape

(2138, 6)

In [31]:
# 독립변수(X, 문제)와 종속변수(y, 정답)
x = df["내용"]
y = df["분류"]

## label one-hot 형태로 만들기

In [54]:
# get_dummies 를 사용하여 label 값을 one-hot 형태로 생성
y_oh = pd.get_dummies(y)

In [32]:
# train_test_split 으로 학습과 예측에 사용할 데이터를 나누기
x_train, x_test, y_train, y_test = train_test_split(x, y_oh, test_size = 0.2, stratify = y_oh, random_state = 12)

In [14]:
okt = Okt()

In [38]:
x_train = x_train.map(lambda x: re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", x))
x_test = x_test.map(lambda x: re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", x))

In [43]:
%%time
x_train_token = x_train.map(lambda x: okt.morphs(x, stem = True))
x_test_token = x_test.map(lambda x: okt.morphs(x, stem = True))

Wall time: 36.7 s


In [47]:
x_train_token

1188    [안전, 관리, 계획, 서, 작, 성, 대상, 은, 건설, 기술, 관리, 법, 제,...
917     [사업, 조정제, 도란, 대기업, 등, 의, 사업, 진출, 로, 중소기업, 의, 경...
631     [조산, 원, 의, 시설, 기준, 은, 입원, 실, 분, 만, 실, 이용, 조, 제...
1316    [귀농, 하다, 하다, 농업, 창업, 및, 주택, 구입, 지, 원, 사업, 은, 어...
1871    [나우, 스타트, 사업, 업무, 개요, 새롭다, 교육, 복지, 의, 기회, 를, 지...
                              ...                        
16      [사업, 의, 주요, 사업, 분야, 에는, 어떻다, 있다, 사업, 분야, 에는, 산...
933     [송파, 뉴스레터, 는, 어떻다, 구성, 되어다, 있다, 돋다, 움, 돋다, 움, ...
1044    [옥외광고, 업자, 교육, 밉다, 이수자, 과태료, 부과, 관련, 옥외광고, 물등,...
212     [서울시, 부동산, 정보, 통합, 열람, 개요, 한번, 의, 검색, 으로, 해당, ...
511     [공공시설, 물, 디자인, 기, 본, 전략, 도, 시공간, 주체, 들, 의, 자유롭...
Name: 내용, Length: 1710, dtype: object

In [48]:
x_train_token = x_train_token.map(lambda x: [i for i in x if len(i) > 1])
x_test_token = x_test_token.map(lambda x: [i for i in x if len(i) > 1])

In [52]:
x_train_token

1188    [안전, 관리, 계획, 대상, 건설, 기술, 관리, 아래, 같다, 시특법, 규정, ...
917     [사업, 조정제, 도란, 대기업, 사업, 진출, 중소기업, 안정, 현저, 하다, 영...
631          [조산, 시설, 기준, 입원, 이용, 실조, 제실, 두다, 경우, 소독, 시설]
1316    [귀농, 하다, 하다, 농업, 창업, 주택, 구입, 사업, 어떻다, 신청, 하나요,...
1871    [나우, 스타트, 사업, 업무, 개요, 새롭다, 교육, 복지, 기회, 지금, 제공,...
                              ...                        
16      [사업, 주요, 사업, 분야, 에는, 어떻다, 있다, 사업, 분야, 에는, 업체, ...
933     [송파, 뉴스레터, 어떻다, 구성, 되어다, 있다, 돋다, 돋다, 돋다, 돋다, 돋...
1044    [옥외광고, 업자, 교육, 밉다, 이수자, 과태료, 부과, 관련, 옥외광고, 물등,...
212     [서울시, 부동산, 정보, 통합, 열람, 개요, 한번, 검색, 으로, 해당, 필지,...
511     [공공시설, 디자인, 전략, 시공간, 주체, 자유롭다, 참여, 살다, 도시, 원천,...
Name: 내용, Length: 1710, dtype: object

## 벡터화
### 토큰화

1. 이 클래스를 사용하면 각 텍스트를 일련의 정수(각 정수는 사전에 있는 토큰의 인덱스임) 또는 단어 수에 따라 각 토큰의 계수가 이진일 수 있는 벡터로 변환하여 텍스트 말뭉치를 벡터화할 수 있습니다.(tf-idf 기반)

2. 매개변수
- num_words
: 단어 빈도에 따라 유지할 최대 단어 수입니다. 가장 일반적인 단어 만 유지됩니다. 
-filters
: 각 요소가 텍스트에서 필터링될 문자인 문자열입니다. 기본값은 문자를 제외한 모든 구두점과 탭 및 줄 바꿈 '입니다.
- lower
: 부울. 텍스트를 소문자로 변환할지 여부입니다.
- split
: str. 단어 분할을 위한 구분 기호입니다.
- char_level
: True이면 모든 문자가 토큰으로 처리됩니다.
- oov_token
: 주어진 경우, 그것은 word_index에 추가되고 text_to_sequence 호출 중에 어휘 밖의 단어를 대체하는 데 사용됩니다.

3. 벡터화 과정
- Tokenizer 인스턴스를 생성
- fit_on_texts와 word_index를 사용하여 key value로 이루어진 딕셔너리를 생성
- texts_to_sequences를 이용하여 text 문장을 숫자로 이루어진 리스트로 변경
- 마지막으로 pad_sequences를 이용하여 리스트의 길이를 통일화

In [55]:
# Tokenizer 는 데이터에 출현하는 모든 단어의 개수를 세고 빈도 수로 정렬해서 
# num_words 에 지정된 만큼만 숫자로 반환하고, 나머지는 0 으로 반환
tokenizer = Tokenizer(oov_token = "<OOV>")

In [56]:
# Tokenizer 에 데이터 실제로 입력합니다.
# fit_on_texts와 word_index를 사용하여 key value로 이루어진 딕셔너리를 생성
tokenizer.fit_on_texts(x_train_token)

In [57]:
# tokenizer의 word_index 속성은 단어와 숫자의 키-값 쌍을 포함하는 딕셔너리를 반환
# 이때, 반환 시 자동으로 소문자로 변환되어 들어가며, 느낌표나 마침표 같은 구두점은 자동으로 제거
# 각 인덱스에 해당하는 단어가 무엇인지 확인
word_to_index  = tokenizer.word_index

In [58]:
word_to_index

{'<OOV>': 1,
 '하다': 2,
 '있다': 3,
 '되다': 4,
 '으로': 5,
 '돋다': 6,
 '경우': 7,
 '에서': 8,
 '시설': 9,
 '사업': 10,
 '신청': 11,
 '관리': 12,
 '지원': 13,
 '이다': 14,
 '또는': 15,
 '하고': 16,
 '주택': 17,
 '센터': 18,
 '서울시': 19,
 '운영': 20,
 '이상': 21,
 '사항': 22,
 '교육': 23,
 '받다': 24,
 '계획': 25,
 '업무': 26,
 '내용': 27,
 '서울': 28,
 '이용': 29,
 '지역': 30,
 '관련': 31,
 '따르다': 32,
 '대상': 33,
 '확인': 34,
 '에는': 35,
 '기준': 36,
 '해당': 37,
 '의하다': 38,
 '규정': 39,
 '홈페이지': 40,
 '설치': 41,
 '건축물': 42,
 '도시': 43,
 '제조': 44,
 '되어다': 45,
 '없다': 46,
 '방법': 47,
 '가능하다': 48,
 '신고': 49,
 '변경': 50,
 '시간': 51,
 '사용': 52,
 '주민': 53,
 '기관': 54,
 '건축': 55,
 '상담': 56,
 '정보': 57,
 '공무원': 58,
 '시행': 59,
 '민원': 60,
 '위원회': 61,
 '도로': 62,
 '건설': 63,
 '서비스': 64,
 '등록': 65,
 '아니다': 66,
 '기간': 67,
 '어떻다': 68,
 '접수': 69,
 '가능': 70,
 '않다': 71,
 '문의': 72,
 '에게': 73,
 '토지': 74,
 '제출': 75,
 '기술': 76,
 '기타': 77,
 '서류': 78,
 '대한': 79,
 '무엇': 80,
 '복지': 81,
 '처리': 82,
 '공사': 83,
 '환경': 84,
 '포함': 85,
 '어린이집': 86,
 '이하': 87,
 '구역': 88,
 '보육': 89,
 '필요하다': 90

In [59]:
tokenizer.word_counts

OrderedDict([('안전', 150),
             ('관리', 570),
             ('계획', 451),
             ('대상', 370),
             ('건설', 259),
             ('기술', 216),
             ('아래', 52),
             ('같다', 110),
             ('시특법', 1),
             ('규정', 332),
             ('의하다', 333),
             ('시설', 901),
             ('물의', 21),
             ('공사', 191),
             ('지하', 22),
             ('굴착', 6),
             ('이상', 476),
             ('미만', 114),
             ('건축물', 324),
             ('또는', 546),
             ('모델링', 7),
             ('해체', 6),
             ('사업', 875),
             ('조정제', 1),
             ('도란', 12),
             ('대기업', 1),
             ('진출', 3),
             ('중소기업', 24),
             ('안정', 24),
             ('현저', 9),
             ('하다', 5510),
             ('영향', 14),
             ('미치다', 9),
             ('우려', 20),
             ('있다', 1814),
             ('경우', 934),
             ('일정', 76),
             ('기간', 255),
             ('인수', 4),
    

In [61]:
total_cnt = len(tokenizer.word_index)
total_cnt

10026

In [64]:
# 단어별 빈도수를 확인
threshold = 2
total_cnt = len(tokenizer.word_index)
rare_cnt = 0
total_freq = 0
rare_freq = 0

for key, value in tokenizer.word_counts.items():
    total_freq += value
    if value < threshold:
        rare_cnt += 1
        rare_freq += value

In [65]:
print("단어 집합의 크기: ", total_cnt)
print("등장 빈도가 %d번 이하인 희귀 단어의 수: %d" % (threshold-1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt) * 100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율: ", (rare_freq / total_freq) * 100)

단어 집합의 크기:  10026
등장 빈도가 1번 이하인 희귀 단어의 수: 3935
단어 집합에서 희귀 단어의 비율: 39.247955316177936
전체 등장 빈도에서 희귀 단어 등장 빈도 비율:  3.2002797703280796


In [66]:
10026 - 3935

6091

In [67]:
tokenizer = Tokenizer(oov_token = "<OOV>", num_words = 6000)

In [68]:
tokenizer.fit_on_texts(x_train_token)

In [69]:
# texts_to_sequences를 이용하여 text 문장을 숫자로 이루어진 리스트로 변경
x_train_seq = tokenizer.texts_to_sequences(x_train_token)
x_test_seq = tokenizer.texts_to_sequences(x_test_token)

## 패딩(Padding)

In [73]:
lengths = np.array([len(x) for x in x_train_seq])

In [74]:
print(np.mean(lengths), np.min(lengths), np.median(lengths), np.max(lengths))

71.90526315789474 3 40.0 2369


In [75]:
np.quantile(lengths, [0.25, 0.5, 0.75, 0.9])

array([ 22.,  40.,  78., 160.])

In [77]:
# 독립변수를 전처리합니다. 
# 문장의 길이가 제각각인 벡터의 크기를 패딩 작업을 통해 나머지 빈 공간을 0으로 채움
# max_length는 패딩의 기준이 됨

x_train_seq = pad_sequences(x_train_seq, truncating = "post", maxlen = 160)
x_test_seq = pad_sequences(x_test_seq, truncating = "post", maxlen = 160)

In [78]:
x_train_seq.shape, x_test_seq.shape

((1710, 160), (428, 160))

## 모델 만들기

In [102]:
model = keras.Sequential()

In [103]:
model.add(keras.layers.Embedding(6000, 128, input_shape = (160,)))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences = True, dropout = 0.2)))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(32, dropout = 0.2)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(16, activation = "relu"))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(3, activation = "softmax"))

## 모델 컴파일

In [104]:
# 여러개 정답 중 하나 맞추는 문제이며, 정답값이 one-hot 형태이기 때문에
# 손실 함수는 categorical_crossentropy를 사용
opt = keras.optimizers.Adam(learning_rate = 0.0005)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics = ["accuracy"])

In [105]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 160, 128)          768000    
                                                                 
 bidirectional_8 (Bidirectio  (None, 160, 128)         98816     
 nal)                                                            
                                                                 
 bidirectional_9 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dropout_8 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 16)                1040      
                                                                 
 dropout_9 (Dropout)         (None, 16)               

In [106]:
es = keras.callbacks.EarlyStopping(patience = 8, restore_best_weights=True)

## 학습

In [107]:
# 모델 학습을 실행
history = model.fit(x_train_seq, y_train, epochs = 100, batch_size = 64, callbacks = [es],
                    validation_split = 0.2)

Epoch 1/100
22/22 [==============================] - 5s 71ms/step - loss: 1.0351 - accuracy: 0.4905 - val_loss: 0.9439 - val_accuracy: 0.5175
Epoch 2/100
22/22 [==============================] - 1s 31ms/step - loss: 0.9358 - accuracy: 0.5190 - val_loss: 0.8771 - val_accuracy: 0.5556
Epoch 3/100
22/22 [==============================] - 1s 30ms/step - loss: 0.8383 - accuracy: 0.6360 - val_loss: 0.7468 - val_accuracy: 0.6930
Epoch 4/100
22/22 [==============================] - 1s 31ms/step - loss: 0.5892 - accuracy: 0.7690 - val_loss: 0.6207 - val_accuracy: 0.7602
Epoch 5/100
22/22 [==============================] - 1s 44ms/step - loss: 0.4417 - accuracy: 0.8465 - val_loss: 0.5989 - val_accuracy: 0.7544
Epoch 6/100
22/22 [==============================] - 1s 30ms/step - loss: 0.3067 - accuracy: 0.8852 - val_loss: 0.5726 - val_accuracy: 0.7836
Epoch 7/100
22/22 [==============================] - 1s 31ms/step - loss: 0.2442 - accuracy: 0.9167 - val_loss: 0.5618 - val_accuracy: 0.8158
Epoch 

## 평가

In [108]:
model.evaluate(x_test_seq, y_test)

14/14 [==============================] - 0s 11ms/step - loss: 0.6525 - accuracy: 0.7804


[0.6525000929832458, 0.7803738117218018]